#### This python notebook is used for exploratory data analysis about the given transactions files in mempool folder and made some modifications to these files

In [21]:
import os
import json
from Helper import *
from Script import *
all_files= list_all_tx() 

In [22]:
scriptpubkey_types=set()

for tx in all_files:
  for section in ["vin", "vout"]:
    for item in tx[section]:
        if section == "vin":     
          scriptpubkey_types.add(item["prevout"]["scriptpubkey_type"])
        else:  
           scriptpubkey_types.add(item["scriptpubkey_type"])
scriptpubkey_types  
          


{'bare-multisig',
 'op_return',
 'p2pkh',
 'p2sh',
 'v0_p2sh_p2wpkh',
 'v0_p2sh_p2wsh',
 'v0_p2wpkh',
 'v0_p2wsh',
 'v1_p2tr'}

In [23]:
# op_return -> 
op_return_in_input=set()
op_return_in_output=set()



for tx in all_files:
  for section in ["vin", "vout"]:
    for item in tx[section]:
        if section == "vin":  
          if (item["prevout"]["scriptpubkey_type"])=='op_return':
             op_return_in_input.add(item["prevout"]["value"])
             
          scriptpubkey_types.add(item["prevout"]["scriptpubkey_type"])
        else:
            if (item["scriptpubkey_type"])=='op_return':
             op_return_in_output.add(item["value"])
               
print(op_return_in_input)
print(op_return_in_output)


set()
{0}


In [24]:
# is 0 value in vin -> 
count=0
for tx in all_files:
  for item in tx["vin"]:            
          if (item["prevout"]["value"])==0:
             count+=1
             
print(count) 



0


### No value 0 in input field


#### Investigating `unknown` keyword in `scriptpubkey-type`:


In [25]:
### First we will check that these keyword are present in both inputs and outputs field or not> 


unknown_keyword= {
   "vin":0,
   "vout":0
}

unique_commands_for_unknown=set()
trial=set()

for tx in all_files:
  for section in ["vin", "vout"]:
    for item in tx[section]:
        if section == "vout" and  item["scriptpubkey_type"]=="unknown":     
          unknown_keyword["vout"]+=1
        if section == "vin" and  item["prevout"]["scriptpubkey_type"]=="unknown":  
            unknown_keyword["vin"]+=1

        #### In some places , "unknown" keyword is written inside the scriptpubkey_type 
        ### -> but it is actually Multisig as shown ->
        
        
        if section == "vout":
           if item["scriptpubkey_type"]=="bare-multisig":
              unique_commands_for_unknown.add(item["scriptpubkey_asm"].split()[-1])
              trial.add(item["scriptpubkey"][-2:])
              

print("unknown keyword present : {}".format(unknown_keyword))   
print("unique_commands for the unknown type at the end {}".format(unique_commands_for_unknown))
print(trial)


unknown keyword present : {'vin': 0, 'vout': 0}
unique_commands for the unknown type at the end {'OP_CHECKMULTISIG'}
{'ae'}


In [26]:
for file in all_files:
    for output in file["vout"]:
        script_type = output["scriptpubkey_type"]
        if script_type == "unknown":
            output["scriptpubkey_type"] = "multisig"


In [27]:
### Thus replace `unknown` keyword to `bare-multisig` scriptpubkey-type

folder_path = "./mempool"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    for output in data["vout"]:
        script_type = output["scriptpubkey_type"]
        if script_type == "unknown":
            output["scriptpubkey_type"] = "bare-multisig"
    
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)


---


#### Version && Locktime(absolute)->


In [28]:
## how many types of version are used ->
version={
    1:0,
    2:0,
    }

seq_for_1={}

locktime_1={
    "Block":set(),
    "Time":set()
}
max_value = (1 << 32) - 1
for file in all_files:
    version[(file["version"])]+=1

    if file["version"]==1:
        for input in file["vin"]:
            if seq_for_1.get(input["sequence"])==None:
                value= input["sequence" ]
                seq_for_1.update({value:0})
            seq_for_1[value]+=1

    locktime_value= file["locktime"]
    if locktime_value<500_000_000:
       locktime_1["Block"].add(locktime_value)
       
    else:
         locktime_1["Time"].add(locktime_value)         

sorted_dict = dict(sorted(seq_for_1.items(), key=lambda item: item[0]))

print(version)
print(sorted_dict)

print("locktime for version 1: {}".format(locktime_1))

# print(version.get("helo"))
print(max(locktime_1["Block"])) 
print(max(locktime_1["Time"])) 





{1: 2259, 2: 5872}
{0: 32, 1: 29, 4443: 475, 4994: 34, 86695: 15, 160346: 418, 1516297: 220, 2559669: 4, 5860418: 8, 6746720: 360, 6777465: 178, 11617556: 5, 11851132: 12, 11910301: 513, 12311610: 220, 13424498: 17, 19526132: 232, 20230627: 3, 21176520: 20, 41617237: 15, 79593512: 13, 132648800: 8, 268435456: 29, 338499383: 21, 357913941: 349, 469337440: 42, 536870911: 442, 651726258: 69, 652251065: 2, 715827882: 57, 724759630: 659, 751167129: 9, 756373883: 8, 910345399: 20, 929525068: 34, 1073741822: 11, 1073745840: 85, 1078771002: 458, 1115905731: 11, 1227143699: 12, 1288490187: 220, 1407676897: 468, 1524582426: 1, 1601075568: 822, 1610612733: 10, 1610613036: 6, 1610616404: 344, 1656665503: 49, 1762298852: 20, 1798317181: 3, 1874281885: 28, 1879051786: 321, 1927488287: 135, 1986185223: 265, 2147483644: 79, 2147483648: 53, 2147483649: 5, 2147483667: 9, 2147483669: 380, 2147483680: 2, 2147483684: 216, 2147483687: 5, 2147483695: 2, 2147483705: 34, 2147483718: 89, 2147483720: 469, 214748

### Observation->

- Two version 1 and 2 are used
- Absolute Locktime given in field in each tx -> is passed at the current -> thus all txs are unlocked to be included in the block (from abosolute locktime perpective)

---


- Check that wether there any tx exists which do not have any input or outputs->
- If txId is of 32 bytes or not.
-


In [29]:
vin_empty=0
vout_empty=0
txid_irregular_length=set()
invalid_vout=0
invalid_value=0
empty_prevout=0
scriptpubkey_empty=0
scriptpubkey_asm_empty=0
wrong_sequence_value_count=0

seq_max_value = 2**32 - 1

for file in all_files:
    if len(file["vin"])==0:
        vin_empty+=1
    if len(file["vout"])==0:
        vout_empty+=1 
    
    for input in file["vin"]:

      if len(input["txid"]) != 64:
        txid_irregular_length.add(file["txid"])

    
      if input["prevout"]["value"] <0 or input["prevout"]["value"] > 2100000000000000:
         invalid_value+=1

      if input["vout"]<0:
         invalid_vout+=1


      if not bool(input["prevout"]):
         empty_prevout+=1


      if len(input["prevout"]["scriptpubkey"]) ==0:
         scriptpubkey_empty+=1

      if len(input["prevout"]["scriptpubkey_asm"]) ==0:
         scriptpubkey_asm_empty+=1   

      if    input["sequence"] > seq_max_value or input["sequence"]<0:
         wrong_sequence_value_count+=1
         
         
    for output in file["vout"]:
       if output["value"] <0 or output["value"] > 2100000000000000:
         invalid_value+=1    


       if len(output["scriptpubkey_asm"]) ==0:
         scriptpubkey_asm_empty+=1    


print("vin empty: {}".format(vin_empty))  
print("vout empty: {}".format(vout_empty)) 
print("txid irregular lenght:{}".format(txid_irregular_length))
print("invalid value :{}".format(invalid_value))
print("invalid vout value count:{}".format(invalid_vout))
print("prevout empty: {}".format(empty_prevout)) 
print("scriptpubkey_empty: {}".format(scriptpubkey_empty)) 
print("scriptpubkey_asm_empty: {}".format(scriptpubkey_asm_empty)) 
print("wrong_sequence_value_count: {}".format(wrong_sequence_value_count)) 




vin empty: 0
vout empty: 0
txid irregular lenght:set()
invalid value :0
invalid vout value count:0
prevout empty: 0
scriptpubkey_empty: 0
scriptpubkey_asm_empty: 0
wrong_sequence_value_count: 0


#### What is difference between scriptpubkey_address and bytes in scriptpubkey_asm ?

- `scriptpubkey_address`-> address each of script
- Bytes in asm -> use that to find the address and then match


##### Basic Sanity check over:-----------------------------------------------------------------------


### Check that whether scriptpubkey and scriptpubkey_asm matches or not ->


Sequence->

- check whether the value is in the range of 4 bytes value or not

- For version 1 -> no meaning of its value
- For version 2 -> check that whether there are tx which enables relative timelock & RBF
- Ignore `Absolute Timelock` -> as all tx in mempool are passed.


- Are there any tx which uses input from the coinbase tx


In [30]:
bool_value=set()
for file in all_files:
    for input in file["vin"]:       
     bool_value.add(input["is_coinbase"])
bool_value    

{False}

## Analysis of txns including `P2SH`->


In [31]:
witness_empty=0
witness_empty_file=[]
redeem_witness_empty=[]
redeem_empty_witness_present=[]
redeem_present_witness_present=0
psh=0
redeem_present_witness_empty= []

for file in all_files:
   
        for input in file['vin']:
         
            if input['prevout']['scriptpubkey_type']== 'p2sh':
               psh+=1
               if input.get("witness")==None:
                    witness_empty+=1
                    witness_empty_file.append(file)
                    if input.get("inner_redeemscript_asm")==None:
                         redeem_witness_empty.append(file)
                         
               if input.get("witness")==None:
                    witness_empty+=1
                    witness_empty_file.append(file)
               if input.get("witness") != None and input.get("inner_redeemscript_asm")==None:
                    redeem_empty_witness_present.append(file)

               if input.get("witness") != None: 
                  if input.get("inner_redeemscript_asm")!=None:
                     redeem_present_witness_present+=1   
               if input.get("witness") == None and input.get("inner_redeemscript_asm")==None:
                    redeem_present_witness_empty.append(file)
                      


# print(witness_empty)                    
                   
# print(witness_empty_file[0])
print(redeem_witness_empty)

print(redeem_empty_witness_present)
print(redeem_present_witness_present)
print(psh)
print(len(redeem_present_witness_empty))


              


[]
[]
0
38
0


##### Observation ->

- If in all tx script type -> there is no mention of `p2sh-p2wkh` and `p2sh-p2wsh` script type.
- Note : `P2sh` -> Total = 3672 , `p2sh` = 38 , nested_p2sh = rest

### Idea to segreate them ->

- `p2sh` -> no witness + yes(inner_redeem_script)
- `p2sh-p2wpkh` -> witness + inner_redeem_script - inner_witness_script
- `p2sh-p2wsh` -> witness + inner_redeem_script + inner_witness_script
- **Note** -> We will change only in vin as in case of vout -> we don't know any more info from the script_pubkey alone


In [32]:
folder_path = "./mempool"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    
        for input in data["vin"]: 
            script_type = input["prevout"]["scriptpubkey_type"]
            if script_type == "p2sh":
                  if input.get("witness") != None and input.get("inner_redeemscript_asm") != None:
                      if input.get("inner_witnessscript_asm") != None:
                         input["prevout"]["scriptpubkey_type"] ="v0_p2sh_p2wsh"
                      else:
                          input["prevout"]["scriptpubkey_type"]="v0_p2sh_p2wpkh"
        with open(file_path, 'w') as f:
          json.dump(data, f, indent=4)
                            

**Check** witness field should not exist and script_sig not empty for legacy address and vice versa for segwit script


- Check whether `parse(sript_pubkey)` == `scriptpubkey_asm` or not ?
- `parse(sript_sig)` == `scriptsig_asm`?
- Find all opcodes used in the script for definign the function



In [33]:
script_pubkey_unmatch=[]
script_sig_unmatch=[]
all_opcodes_used = set() 
for file_index, file in enumerate(all_files):
    for section in ["vout","vin"]:
        for input_index, input in enumerate(file[section]):
            scriptpubkey_asm=""
            parsed_script_pubkey=""
            parsed_script_sig=""
            scriptsig_asm=""
            if section == "vin":
                parsed_script_pubkey=  Script.parse(input["prevout"]["scriptpubkey"])
                parsed_script_sig= Script.parse(input["scriptsig"])
                scriptpubkey_asm= input["prevout"]["scriptpubkey_asm"]
                scriptsig_asm= input["scriptsig_asm"]
            else:
                parsed_script_pubkey= Script.parse(input["scriptpubkey"])
                scriptpubkey_asm= input["scriptpubkey_asm"]

            if str(parsed_script_pubkey)!= scriptpubkey_asm:
                script_pubkey_unmatch.append([file_index,section])    
            if str(parsed_script_sig) !=  scriptsig_asm:
                script_sig_unmatch.append([file_index,section,input_index]) 
            # finding all the opcodes used in the script ->     
            for script_type in [scriptpubkey_asm,scriptsig_asm]:
                for word in script_type.split():
                    if word[:2] == "OP":
                        all_opcodes_used.add(word)




print(len(script_pubkey_unmatch))
print("script_pubkey_unmatch : {}".format(script_pubkey_unmatch))
print("script_sig_unmatch:{}".format(script_sig_unmatch))  
print("all opcodes used : {}".format(all_opcodes_used))              

                


53456
script_pubkey_unmatch : [[0, 'vout'], [0, 'vin'], [1, 'vout'], [1, 'vin'], [1, 'vin'], [2, 'vout'], [2, 'vout'], [2, 'vin'], [3, 'vout'], [3, 'vout'], [3, 'vin'], [4, 'vout'], [4, 'vout'], [4, 'vin'], [4, 'vin'], [5, 'vout'], [5, 'vin'], [6, 'vout'], [6, 'vout'], [6, 'vin'], [7, 'vout'], [7, 'vin'], [8, 'vout'], [8, 'vin'], [9, 'vout'], [9, 'vout'], [9, 'vin'], [10, 'vout'], [10, 'vout'], [10, 'vin'], [11, 'vout'], [11, 'vin'], [12, 'vout'], [12, 'vout'], [12, 'vout'], [12, 'vout'], [12, 'vin'], [12, 'vin'], [13, 'vout'], [13, 'vin'], [14, 'vout'], [14, 'vin'], [14, 'vin'], [15, 'vout'], [15, 'vin'], [16, 'vout'], [16, 'vin'], [17, 'vout'], [17, 'vin'], [18, 'vout'], [18, 'vin'], [19, 'vout'], [19, 'vin'], [20, 'vout'], [20, 'vin'], [21, 'vout'], [21, 'vin'], [21, 'vin'], [21, 'vin'], [21, 'vin'], [22, 'vout'], [22, 'vin'], [23, 'vout'], [23, 'vin'], [24, 'vout'], [24, 'vout'], [24, 'vin'], [25, 'vout'], [25, 'vin'], [26, 'vout'], [26, 'vout'], [26, 'vin'], [26, 'vin'], [26, 'vin

#### Observation ->

- `Scriptpubkey` matches with `Scriptpubkey_asm` + script_case for all txn
- Total 41 opcodes are used -> `all_opcodes_used`


##### Check whether if the scriptpubkey type is legacy then it should not contain witness field and segwit script should contain this field

##### Check `OP_RETURN` scripttype should come where `value = 0`


##### Check scriptpubkey corresponds to scriptpubkey_type:


##### Check if for any tx -> fee <= 0 -> if so then tx is invalid


In [34]:
legacy_contain_witness_vice_versa=[]
op_return_then_zero_value=[]
op_return_no=0

# scripttype with scriptpubkey
scriptpub_matches_with_scriptpubtype=set()

# collect all tx whose fee<= 0
invalid_tx_with_fee_lessthan_zero=[]
for file in all_files:
    for section in ["vin","vout"]:
     if section == "vin":
      for input in file["vin"]:
        script_type= input["prevout"]["scriptpubkey_type"]
        scriptpubkey_asm= input["prevout"]["scriptpubkey_asm"]
        ans=  Script.is_scriptpubkey_match_scripttype( scriptpubkey_asm,script_type)
        scriptpub_matches_with_scriptpubtype.add(ans)


        if input["prevout"]["scriptpubkey_type"] in ["p2sh","p2pkh","bare-multisig"]:
            if input.get("witness")!=None:
                legacy_contain_witness_vice_versa.append(file)
        else:
            if  input.get("witness") == None:
                legacy_contain_witness_vice_versa.append(file) 
     else:
        for output in file["vout"]:
           if output["scriptpubkey_type"]=="op_return":
              op_return_no+=1
              if output["value"]==0:
                 op_return_then_zero_value.append(file)
              
           
        
                
                   
print(legacy_contain_witness_vice_versa)
print(op_return_no)
print(len(op_return_then_zero_value))
scriptpub_matches_with_scriptpubtype

[]
117
117


{True}

**Observation :**

- legacy script not have witness & segwit script have witness field -> all ok
- Wherever scripttype is `op_return` -> value =0
- For every tx -> given scriptpubkey is corresponding to given script_type -> all ok

# ---------------------------------------------------------------------